In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

results = pd.read_csv (r"results.csv", encoding= 'unicode_escape')

In [2]:
results = results.dropna()

In [3]:
results = results.drop(['HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC',
       'HF', 'AF', 'HY', 'AY', 'HR', 'AR'], axis=1)

In [4]:
results.head()

,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR
2824,2000-01,2000-08-19T00:00:00Z,Charlton,Man City,4,0,H
2825,2000-01,2000-08-19T00:00:00Z,Chelsea,West Ham,4,2,H
2826,2000-01,2000-08-19T00:00:00Z,Coventry,Middlesbrough,1,3,A
2827,2000-01,2000-08-19T00:00:00Z,Derby,Southampton,2,2,D
2828,2000-01,2000-08-19T00:00:00Z,Leeds,Everton,2,0,H


In [5]:
results = results.rename(columns={'DateTime': 'Date'})

In [6]:
seasons = {}
seasons = results["Season"].unique()
seasons

array(['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06',
       '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12',
       '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18',
       '2018-19', '2019-20', '2020-21', '2021-22'], dtype=object)

In [7]:
for i in seasons:
    print(len(results[results['Season'] == i]))

380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
380
70


In [8]:
results = results[results.Season != "2021-22"]

In [9]:
seasons = results["Season"].unique()
seasons

array(['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06',
       '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12',
       '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18',
       '2018-19', '2019-20', '2020-21'], dtype=object)

In [10]:
results.tail()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
10799,2020-21,2021-05-23T16:00:00Z,Liverpool,Crystal Palace,2,0,H
10800,2020-21,2021-05-23T16:00:00Z,Man City,Everton,5,0,H
10801,2020-21,2021-05-23T16:00:00Z,Sheffield United,Burnley,1,0,H
10802,2020-21,2021-05-23T16:00:00Z,West Ham,Southampton,3,0,H
10803,2020-21,2021-05-23T16:00:00Z,Wolves,Man United,1,2,A


In [11]:
results.shape

(7980, 7)

In [12]:
len(seasons)

21

In [13]:
for i in range(len(seasons)):
    seasons[i] = pd.DataFrame(results[results['Season'] == seasons[i]])

seasons

array([       Season                  Date       HomeTeam       AwayTeam  FTHG  FTAG  \
       2824  2000-01  2000-08-19T00:00:00Z       Charlton       Man City     4     0
       2825  2000-01  2000-08-19T00:00:00Z        Chelsea       West Ham     4     2
       2826  2000-01  2000-08-19T00:00:00Z       Coventry  Middlesbrough     1     3
       2827  2000-01  2000-08-19T00:00:00Z          Derby    Southampton     2     2
       2828  2000-01  2000-08-19T00:00:00Z          Leeds        Everton     2     0
       ...       ...                   ...            ...            ...   ...   ...
       3199  2000-01  2001-05-19T00:00:00Z       Man City        Chelsea     1     2
       3200  2000-01  2001-05-19T00:00:00Z  Middlesbrough       West Ham     2     1
       3201  2000-01  2001-05-19T00:00:00Z      Newcastle    Aston Villa     3     0
       3202  2000-01  2001-05-19T00:00:00Z    Southampton        Arsenal     3     2
       3203  2000-01  2001-05-19T00:00:00Z      Tottenham     

In [14]:
# Gets the goals scored agg arranged by teams and matchweek
def get_goals_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)

     # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

In [15]:
# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)

     # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

In [16]:
def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat

In [17]:
for i in range(len(seasons)):
    get_gss(seasons[i])

In [18]:
seasons[15]

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC
8524,2015-16,2015-08-08T00:00:00Z,Bournemouth,Aston Villa,0,1,A,0,0,0,0
8525,2015-16,2015-08-08T00:00:00Z,Chelsea,Swansea,2,2,D,0,0,0,0
8526,2015-16,2015-08-08T00:00:00Z,Everton,Watford,2,2,D,0,0,0,0
8527,2015-16,2015-08-08T00:00:00Z,Leicester,Sunderland,4,2,H,0,0,0,0
8528,2015-16,2015-08-08T00:00:00Z,Man United,Tottenham,1,0,H,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8899,2015-16,2016-05-15T00:00:00Z,Stoke,West Ham,2,1,H,39,64,54,49
8900,2015-16,2016-05-15T00:00:00Z,Swansea,Man City,1,1,D,41,70,51,40
8901,2015-16,2016-05-15T00:00:00Z,Watford,Sunderland,2,2,D,38,46,48,60
8902,2015-16,2016-05-15T00:00:00Z,West Brom,Liverpool,1,1,D,33,62,47,49


In [19]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points

In [20]:
def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

In [21]:
def get_agg_points(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

In [22]:
for i in range(len(seasons)):
    get_agg_points(seasons[i])

In [23]:
seasons[20]

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP
10424,2020-21,2020-09-12T12:30:00Z,Fulham,Arsenal,0,3,A,0,0,0,0,0,0
10425,2020-21,2020-09-12T15:00:00Z,Crystal Palace,Southampton,1,0,H,0,0,0,0,0,0
10426,2020-21,2020-09-12T17:30:00Z,Liverpool,Leeds,4,3,H,0,0,0,0,0,0
10427,2020-21,2020-09-12T20:00:00Z,West Ham,Newcastle,0,2,A,0,0,0,0,0,0
10428,2020-21,2020-09-13T14:00:00Z,West Brom,Leicester,0,3,A,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,2020-21,2021-05-23T16:00:00Z,Liverpool,Crystal Palace,2,0,H,66,41,42,64,66,44
10800,2020-21,2021-05-23T16:00:00Z,Man City,Everton,5,0,H,78,47,32,43,83,59
10801,2020-21,2021-05-23T16:00:00Z,Sheffield United,Burnley,1,0,H,19,33,63,54,20,39
10802,2020-21,2021-05-23T16:00:00Z,West Ham,Southampton,3,0,H,59,47,47,65,62,43


In [24]:
def get_form(playing_stat,num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

In [25]:
def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent

        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h                 
    playing_stat['AM' + str(num)] = a

    
    return playing_stat

In [26]:
def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics    

In [27]:
for i in range(len(seasons)):
    add_form_df(seasons[i])

In [28]:
seasons[20]

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,...,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
10424,2020-21,2020-09-12T12:30:00Z,Fulham,Arsenal,0,3,A,0,0,0,...,M,M,M,M,M,M,M,M,M,M
10425,2020-21,2020-09-12T15:00:00Z,Crystal Palace,Southampton,1,0,H,0,0,0,...,M,M,M,M,M,M,M,M,M,M
10426,2020-21,2020-09-12T17:30:00Z,Liverpool,Leeds,4,3,H,0,0,0,...,M,M,M,M,M,M,M,M,M,M
10427,2020-21,2020-09-12T20:00:00Z,West Ham,Newcastle,0,2,A,0,0,0,...,M,M,M,M,M,M,M,M,M,M
10428,2020-21,2020-09-13T14:00:00Z,West Brom,Leicester,0,3,A,0,0,0,...,M,M,M,M,M,M,M,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,2020-21,2021-05-23T16:00:00Z,Liverpool,Crystal Palace,2,0,H,66,41,42,...,W,L,W,W,W,L,W,W,D,L
10800,2020-21,2021-05-23T16:00:00Z,Man City,Everton,5,0,H,78,47,32,...,L,W,W,L,L,D,W,W,W,L
10801,2020-21,2021-05-23T16:00:00Z,Sheffield United,Burnley,1,0,H,19,33,63,...,L,L,W,L,L,W,L,L,W,W
10802,2020-21,2021-05-23T16:00:00Z,West Ham,Southampton,3,0,H,59,47,47,...,W,L,D,W,L,W,W,L,L,D


In [29]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

In [30]:
for i in range(len(seasons)):
    seasons[i] = seasons[i][cols]

In [31]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

In [32]:
for i in range(len(seasons)):
    seasons[i] = get_mw(seasons[i])

In [33]:
final_df = pd.concat(seasons)

In [34]:
final_df.shape

(7980, 23)

In [ ]:
final_df.columns

In [35]:
# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

In [36]:
final_df['HTFormPtsStr'] = final_df['HM1'] + final_df['HM2'] + final_df['HM3'] + final_df['HM4'] + final_df['HM5']
final_df['ATFormPtsStr'] = final_df['AM1'] + final_df['AM2'] + final_df['AM3'] + final_df['AM4'] + final_df['AM5']

final_df['HTFormPts'] = final_df['HTFormPtsStr'].apply(get_form_points)
final_df['ATFormPts'] = final_df['ATFormPtsStr'].apply(get_form_points)

In [37]:
# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0

def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0

In [38]:
final_df['HTWinStreak3'] = final_df['HTFormPtsStr'].apply(get_3game_ws)
final_df['HTWinStreak5'] = final_df['HTFormPtsStr'].apply(get_5game_ws)
final_df['HTLossStreak3'] = final_df['HTFormPtsStr'].apply(get_3game_ls)
final_df['HTLossStreak5'] = final_df['HTFormPtsStr'].apply(get_5game_ls)

final_df['ATWinStreak3'] = final_df['ATFormPtsStr'].apply(get_3game_ws)
final_df['ATWinStreak5'] = final_df['ATFormPtsStr'].apply(get_5game_ws)
final_df['ATLossStreak3'] = final_df['ATFormPtsStr'].apply(get_3game_ls)
final_df['ATLossStreak5'] = final_df['ATFormPtsStr'].apply(get_5game_ls)

final_df.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'MW', 'HTFormPtsStr', 'ATFormPtsStr',
       'HTFormPts', 'ATFormPts', 'HTWinStreak3', 'HTWinStreak5',
       'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3', 'ATWinStreak5',
       'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [45]:
# Get Goal Difference
final_df['HTGD'] = final_df['HTGS'] - final_df['HTGC']
final_df['ATGD'] = final_df['ATGS'] - final_df['ATGC']

# Diff in points
final_df['DiffPts'] = final_df['HTP'] - final_df['ATP']
final_df['DiffFormPts'] = final_df['HTFormPts'] - final_df['ATFormPts']

In [46]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
final_df.MW = final_df.MW.astype(float)

for col in cols:
    final_df[col] = final_df[col] / final_df.MW

In [49]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
    
final_df['FTR'] = final_df.FTR.apply(only_hw)

In [50]:
final_df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak3,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts
2824,2000-08-19T00:00:00Z,Charlton,Man City,4,0,H,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
2825,2000-08-19T00:00:00Z,Chelsea,West Ham,4,2,H,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
2826,2000-08-19T00:00:00Z,Coventry,Middlesbrough,1,3,NH,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
2827,2000-08-19T00:00:00Z,Derby,Southampton,2,2,NH,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
2828,2000-08-19T00:00:00Z,Leeds,Everton,2,0,H,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,2021-05-23T16:00:00Z,Liverpool,Crystal Palace,2,0,H,66,41,42,64,...,0,0,0,0,0,0,0.631579,-0.605263,0.578947,0.184211
10800,2021-05-23T16:00:00Z,Man City,Everton,5,0,H,78,47,32,43,...,0,0,0,0,0,0,1.210526,0.105263,0.631579,0.052632
10801,2021-05-23T16:00:00Z,Sheffield United,Burnley,1,0,H,19,33,63,54,...,0,0,0,0,0,0,-1.157895,-0.552632,-0.500000,0.000000
10802,2021-05-23T16:00:00Z,West Ham,Southampton,3,0,H,59,47,47,65,...,0,0,0,0,0,0,0.315789,-0.473684,0.500000,0.000000
